<a href="https://colab.research.google.com/github/iKiYo/Test-Code-of-DeepKT-TF2/blob/master/my_DKT_%2B_tempo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download assistment 2012 dataset

#### use edudata, a downloader of IST educational datasets

In [ ]:
!pip install EduData

In [ ]:
!edudata ls

In [ ]:
!edudata download assistment-2012-2013-non-skill

#### Use stored data in Gdrive

In [ ]:
import pandas as pd
assistment_2012_dataset = pd.read_csv("/content/drive/My Drive/master thesis/Datasets/assistment_dataset/2012-2013-data-with-predictions-4-final.csv")

### Data statistic and preprocessing of assistment_2012, which contains timestamps of attempts

In [ ]:
print(len(assistment_2012_dataset))
assistment_2012_dataset.head(1)
print(list(assistment_2012_dataset.columns.values)) # get the list of column header

##### XXX: skill_id has more types than skill(name)??

In [ ]:
print( assistment_2012_dataset.skill_id.unique())
print( assistment_2012_dataset.skill.unique())

In [ ]:
print( assistment_2012_dataset[['user_id', 'skill', 'skill_id','problem_id','assignment_id','assistment_id']].nunique())

#### try out drop nan and check skill

In [ ]:
assistment_2012_dataset_selected = pd.read_csv(
    "/content/drive/My Drive/master thesis/Datasets/assistment_dataset/2012-2013-data-with-predictions-4-final.csv",
    usecols=['problem_log_id', 'skill', 'problem_id', 'user_id', 'assignment_id', 'assistment_id', 'start_time', 'end_time', 'correct', 'attempt_count', 'skill_id'])
print(len(assistment_2012_dataset_selected))
assistment_2012_dataset_selected.head(1)

In [ ]:
df = assistment_2012_dataset_selected[['skill', 'skill_id']].dropna()

In [ ]:
df = df.astype({'skill_id': 'int32'})

In [ ]:
df.nunique()

#### process timestamp columns

In [ ]:
# sort by endtime
selected_df =selected_df.sort_values(by=['end_time'])
selected_df

In [ ]:
counts = selected_df['end_time'].value_counts()
dub_time_df= selected_df[~selected_df['end_time'].isin(counts[counts < 2].index)]
dub_time_df

In [ ]:
dub_time_df[dub_time_df['end_time'] == '2012-10-12 13:41:00.974']

In [ ]:
dub_time_df['end_time'].value_counts()

#### (drop NaN ) make a complete and minimum csv dataset for experiments

In [ ]:
import pandas as pd 
assist2012_preprocessed = pd.read_csv(
    "/content/drive/My Drive/master thesis/Datasets/assistment_dataset/2012-2013-data-with-predictions-4-final.csv",
    usecols=['assistment_id', 'user_id', 'start_time', 'end_time', 'correct', 'attempt_count', 'skill_id', 'skill'])
print(len(assist2012_preprocessed))
assist2012_preprocessed.head(1)

In [ ]:
df = assist2012_preprocessed
print(len(df))
#df = df.drop_duplicates(subset=['problem_id','user_id','attempt_count'])
df = df.drop_duplicates()
print(len(df))
df = df.dropna(subset=['skill_id', 'skill' ])
# change 'correct' column into a boolean format as recommended in ASSIST dataset website
#df['correct'] = (df['correct'] >= 1).astype(int)
df = df[df["correct"].isin([0, 1])]
df = df.astype({'skill_id': 'int32'})
print(len(df))
df.nunique()

##### WARNING: 70% of attempts(labels) are correct

In [ ]:
df.correct.value_counts(normalize=True)

In [ ]:
df

In [ ]:
# remove users with a sequence length less than three
counts = df['user_id'].value_counts()
selected_df= df[~df['user_id'].isin(counts[counts < 3].index)]
print(len(selected_df))
selected_df.nunique()

In [ ]:
selected_df.drop_duplicates()

In [ ]:
selected_df['user_id'].value_counts()

In [ ]:
tdf = selected_df[selected_df['user_id'] == 221324]
#selected_df["timestamp"] = pd.to_datetime(["end_time"])

In [ ]:
skill_counts = selected_df['skill_id'].value_counts()

In [ ]:
skill_counts

In [ ]:
len(counts)

In [ ]:
len(selected_df)

In [ ]:
selected_df.nunique()

In [ ]:
df[df['attempt_count'] == 0]

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
import argparse
import os

data_name = "/content/drive/My Drive/master thesis/Datasets/assistment_dataset/2012-2013-data-with-predictions-4-final.csv"
min_interactions_per_user = 0
remove_nan_skills = True


"""Preprocess ASSISTments dataset.

Arguments:
    data_name: "assistments09", "assistments12", "assistments15" or "assistments17"
    min_interactions_per_user (int): minimum number of interactions per student
    remove_nan_skills (bool): if True, remove interactions with no skill tag
Outputs:
    df (pandas DataFrame): preprocessed ASSISTments dataset with user_id, item_id,
        timestamp and correct features
    Q_mat (item-skill relationships sparse array): corresponding q-matrix
"""
#data_path = os.path.join("data", data_name)
#df = pd.read_csv(os.path.join(data_path, "data.csv"), encoding="ISO-8859-1")
df = pd.read_csv(os.path.join(data_name))
data_name = "assistments12"

In [ ]:
df = df.rename(columns={"problem_id": "item_id"})
df["timestamp"] = pd.to_datetime(df["start_time"])
print(df.dtypes)

In [ ]:
df["timestamp"] = df["timestamp"] - df["timestamp"].min()

In [ ]:
df["timestamp"] = df["timestamp"].apply(lambda x: x.total_seconds()).astype(np.int64)

In [ ]:
# Sort data temporally
if data_name in ["assistments12", "assistments17"]:
    df.sort_values(by="timestamp", inplace=True)
elif data_name == "assistments09":
    df.sort_values(by="order_id", inplace=True)
elif data_name == "assistments15":
    df.sort_values(by="log_id", inplace=True)

# Filter too short sequences
df = df.groupby("user_id").filter(lambda x: len(x) >= min_interactions_per_user)

# Remove continuous outcomes
df = df[df["correct"].isin([0, 1])]
df["correct"] = df["correct"].astype(np.int32)

# Filter nan skills
if remove_nan_skills:
    df = df[~df["skill_id"].isnull()]
else:
    df.ix[df["skill_id"].isnull(), "skill_id"] = -1

df["user_id"] = np.unique(df["user_id"], return_inverse=True)[1]
df["item_id"] = np.unique(df["item_id"], return_inverse=True)[1]
df["skill_id"] = np.unique(df["skill_id"], return_inverse=True)[1]

# Build Q-matrix
Q_mat = np.zeros((len(df["item_id"].unique()), len(df["skill_id"].unique())))
for item_id, skill_id in df[["item_id", "skill_id"]].values:
    Q_mat[item_id, skill_id] = 1



df = df[['user_id', 'item_id', 'timestamp', 'correct']]
df.reset_index(inplace=True, drop=True)


In [ ]:
data_path = "./"

# Save data
sparse.save_npz(os.path.join(data_path, "q_mat.npz"), sparse.csr_matrix(Q_mat))
df.to_csv(os.path.join(data_path, "preprocessed_data.csv"), sep="\t", index=False)

In [ ]:
pre_df = pd.read_csv("/content/preprocessed_data.csv", sep="\t")
pre_df.nunique()


In [ ]:
import random
from random import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import torch
from torch.nn.utils.rnn import pad_sequence


def set_random_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    
    
def get_data(df, train_split=0.8):
    num_items = df["item_id"].nunique()
    data = [(torch.tensor(u_df["item_id"].values, dtype=torch.long),
             torch.tensor(u_df["correct"].values, dtype=torch.long))
            for _, u_df in df.groupby("user_id")]
    data = [(torch.cat((torch.zeros(1, dtype=torch.long), item_ids + labels * num_items + 1))[:-1], item_ids, labels)
            for (item_ids, labels) in data]
    shuffle(data)

    # Train-test split across users
    train_size = int(train_split * len(data))
    train_data, val_data = data[:train_size], data[train_size:]
    return train_data, val_data


def prepare_batches(data, batch_size):
    """Prepare batches grouping padded sequences.
    
    Arguments:
        data (list of tuples of torch Tensor)
        batch_size (int): number of sequences per batch
        
    Output:
        batches (list of tuples of torch Tensor)
    """
    shuffle(data)
    batches = []

    for k in range(0, len(data), batch_size):
        batch = data[k:k + batch_size]
        inputs, item_ids, labels = zip(*batch)

        inputs = pad_sequence(inputs, batch_first=True, padding_value=0)     # Pad with 0
        item_ids = pad_sequence(item_ids, batch_first=True, padding_value=0) # Don't care
        labels = pad_sequence(labels, batch_first=True, padding_value=-1)    # Pad with -1

        batches.append([inputs, item_ids, labels])
        
    return batches


def compute_auc(preds, item_ids, labels):
    labels = labels.view(-1)
    item_ids = item_ids.view(-1)[labels >= 0]
    preds = preds.view(-1, preds.shape[-1])[labels >= 0]
    preds = preds[torch.arange(preds.shape[0]), item_ids]
    labels = labels[labels >= 0].float()

    if len(torch.unique(labels)) == 1: # Only one class
        auc = accuracy_score(labels, preds.round())
    else:
        auc = roc_auc_score(labels, preds)
    return auc


def compute_loss(preds, item_ids, labels, criterion):
    labels = labels.view(-1)
    item_ids = item_ids.view(-1)[labels >= 0]
    preds = preds.view(-1, preds.shape[-1])[labels >= 0]
    preds = preds[torch.arange(preds.shape[0]), item_ids]
    labels = labels[labels >= 0].float()
    return criterion(preds, labels)

#### sample data for debug

In [ ]:
# data dependent parameters
num_student = 1000
num_skills = 5 #M
max_sequence_length =  6 # T

In [ ]:
# q question ids
raw_inputs = np.array([
    [1, 3, 4],
   [3, 4, 2, 1, 2],
   [3, 1, 1, 4, 5, 2],
])
# a answer or attempts on the questions
raw_labels =  np.array([
    [1, 1, 1],
   [0, 0, 0, 1, 1],
   [0, 1, 1, 1, 0, 1]
])



print(F"raw_inputs \n{raw_inputs}")
print(F"raw_labels \n{raw_labels}")


##### tf.Tensor/ndarray format data preprocessing 

In [ ]:
# padded_q
# padding is -1 and adjust indeces of tensors
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, maxlen=max_sequence_length, padding="post", value=0
)
# Warning: supposing the question ids 1> , 
padded_inputs = padded_inputs - 1
print(F"padded_inputs :\n{padded_inputs}")

# padded_a
# padding is 1 to set the padding of x_t to -1
padded_labels= tf.keras.preprocessing.sequence.pad_sequences(
    raw_labels, maxlen=max_sequence_length, padding="post", value=1
)
padded_labels=tf. convert_to_tensor(padded_labels)
print(F"padded_labels:\n {padded_labels}") # q_t
print(padded_labels.dtype)
print()

"""
M to 2M vector
binary => correct columns + incorrect columns 
"""
# raw_x_t = M*(1 - a_t) + q_t 
signaled_input = num_skills * ( 1 - padded_labels) + padded_inputs
print(F"signaled_input:\n{signaled_input}")
print()

# x_t
one_hot_input = tf.one_hot(signaled_input, depth=num_skills*2, axis=-1)
print(F"x_t sample:\n{one_hot_input.numpy()[1]}")
print(one_hot_input.numpy().shape)
print()

# delta(q_t)
one_hot_q_t = tf.one_hot(padded_inputs, depth=num_skills, axis=-1)
print(F"delta(q_t) sample:\n{one_hot_q_t.numpy()[1]}")
one_hot_q_t.numpy().shape

In [ ]:
signaled_sample[0][0]

In [ ]:
for i in range(len(signaled_sample)):
  for k in range(len(padded_sample[i])):
      print(signaled_sample[i][k])
      print(len(signaled_sample[i][k]), len(signaled_sample[i][k]))

In [ ]:
q, x, delta_q = signaled_dataset

In [ ]:
u_df = df.groupby('user_id')
u_df.count()['timestamp'].median()

#### check median of attempts length(T)

### Complete preprocess

###### preprocess with pandas dataframe

In [ ]:
import pandas as pd 

assist2012_preprocessed = pd.read_csv(
    "/content/drive/My Drive/master thesis/Datasets/assistment_dataset/2012-2013-data-with-predictions-4-final.csv",
    usecols=['user_id', 'skill_id', 'correct', 'end_time', 'attempt_count'])

In [53]:
print(len(assist2012_preprocessed))
assist2012_preprocessed.head(1)

6123270


,user_id,end_time,correct,attempt_count,skill_id
0,61394,2012-09-28 15:11:36.856,1.0,1,NaN


In [60]:
df = assist2012_preprocessed

# drop NaN skill_ids 
df = df.dropna(subset = ['skill_id'])
# or fill NaN
#df = df.fillna(value={'skill_id': 999})
#df = df.astype({'skill_id': 'int32'})
print(len(df))
df.nunique()
df

2711813


,user_id,end_time,correct,attempt_count,skill_id
1,61394,2012-10-09 11:02:13.182,1.0,1,54.0
2,61394,2013-03-07 10:53:28.661,0.0,1,279.0
3,61394,2013-08-20 19:55:21.753,1.0,1,79.0
5,76592,2012-12-12 21:01:07.536,1.0,1,86.0
7,78401,2012-10-16 10:31:55.445,1.0,1,311.0
...,...,...,...,...,...
6123259,227693,2013-08-29 13:20:52.98,1.0,1,94.0
6123260,227788,2013-08-29 16:19:08.388,1.0,1,75.0
6123262,227839,2013-08-31 15:15:08.024,1.0,1,34.0
6123264,227908,2013-08-31 00:25:16.236,0.0,3,67.0


In [61]:
df = df[df["correct"].isin([0, 1])]
df['correct'] = (df['correct'] >= 1).astype(int)

df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,end_time,correct,attempt_count,skill_id
1,61394,2012-10-09 11:02:13.182,1,1,54.0
2,61394,2013-03-07 10:53:28.661,0,1,279.0
3,61394,2013-08-20 19:55:21.753,1,1,79.0
5,76592,2012-12-12 21:01:07.536,1,1,86.0
7,78401,2012-10-16 10:31:55.445,1,1,311.0
...,...,...,...,...,...
6123259,227693,2013-08-29 13:20:52.98,1,1,94.0
6123260,227788,2013-08-29 16:19:08.388,1,1,75.0
6123262,227839,2013-08-31 15:15:08.024,1,1,34.0
6123264,227908,2013-08-31 00:25:16.236,0,3,67.0


In [62]:
# delete user with only one attempt
counts = df['user_id'].value_counts()
df= df[~df['user_id'].isin(counts[counts < 2].index)]

# print(selected_df['user_id'].value_counts())
max_sequence_length=  df['user_id'].value_counts().max()
# print(max_sequence_length)
# print(selected_df['skill_id'].value_counts())
# sample entry
# df[df['user_id'] == 221324]
len(df.drop_duplicates()), len(df)

2710702

In [64]:
# sort by endtime(timestamp)
df =df.sort_values(by=['end_time'])
df

,user_id,end_time,correct,attempt_count,skill_id
2012329,169036,2012-09-01 00:01:48.186552,1,1,81.0
1582259,172027,2012-09-01 00:02:06.478,0,3,358.0
3961249,169036,2012-09-01 00:03:02.788824,0,1,81.0
4703521,172027,2012-09-01 00:03:53.329,1,1,358.0
6065676,169036,2012-09-01 00:04:23.367864,0,1,81.0
...,...,...,...,...,...
752604,225337,2013-10-06 20:49:25.159,0,6,312.0
2057397,218970,2013-10-08 16:56:22.989,0,1,378.0
3288044,218970,2013-10-08 17:25:11.741,0,1,346.0
1817838,218970,2013-10-08 17:28:21.378,0,2,378.0


In [65]:
# enumerate and renumber the remain student and skill IDs
df['user_id'], _ = pd.factorize(df['user_id'], sort=False)
df['skill_id'], _ = pd.factorize(df['skill_id'], sort=False)

['user_id', 'end_time', 'correct', 'attempt_count', 'skill_id']

In [72]:
# get N, M, T
num_students, num_skills, _ = df.nunique()
assert max_sequence_length, df['user_id'].value_counts().max()

print(num_students, num_skills, max_sequence_length)

28118 265 2089


In [119]:
# Cross skill id with answer to form a synthetic feature
df['x'] = df['skill_id'] * 2 + df['correct']

In [123]:
df.to_csv("/content/drive/My Drive/master thesis/Datasets/assistment_dataset/assist12_4cols_noNaNskill.csv", index=False)

###### preprocess in the tf.data format

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.test.is_gpu_available())

In [5]:
df = pd.read_csv("/content/drive/My Drive/master thesis/Datasets/assistment_dataset/assist12_4cols_noNaNskill.csv")

In [ ]:
# get N, M, T
num_students, num_skills, _, _ = df.nunique()
max_sequence_length=  df['user_id'].value_counts().max()
print(num_students, num_skills, max_sequence_length)

In [15]:
batch_size = 25

In [16]:
seq = df.groupby('user_id').apply(
    lambda r: (
        r['x'].values[:-1],
        r['skill_id'].values[1:],
        r['correct'].values[1:],
    )
)

In [ ]:
assert num_students, len(seq)
seq[0]

In [17]:
# Step 5 - Get Tensorflow Dataset
dataset = tf.data.Dataset.from_generator(
    generator=lambda: seq,
    output_types=(tf.int32, tf.int32, tf.int32)
)

In [18]:
dataset.shuffle(buffer_size=num_students)

<ShuffleDataset shapes: (<unknown>, <unknown>, <unknown>), types: (tf.int32, tf.int32, tf.int32)>

In [19]:
transformed_dataset  = dataset.map(
    lambda feat, skill, label: (
        tf.one_hot(feat, depth=num_skills*2), # x
        tf.one_hot(skill, depth=num_skills, axis=-1), # q
        tf.expand_dims(label, -1) # a 
    )
)

In [ ]:
padded_sample = list(transformed_dataset.take(3).as_numpy_iterator())
for i in range(3):
  print(padded_sample[0][i].T)
  print(np.array(padded_sample[0][i].T).shape)

In [20]:
padding_length = max_sequence_length
padded_dataset = transformed_dataset.padded_batch(
        batch_size=batch_size,
        padding_values=(0.,0.,-1),
        padded_shapes=([padding_length, 2*num_skills], [padding_length, num_skills], [padding_length, 1]),
        drop_remainder=True
    )
padded_dataset

<PaddedBatchDataset shapes: ((25, 2089, 530), (25, 2089, 265), (25, 2089, 1)), types: (tf.float32, tf.float32, tf.int32)>

In [21]:
# to dict dataset
input_label_dataset = padded_dataset.map(
        lambda x, delta_q, a : (
             {"x" : x,
              "q" : delta_q},
             { "outputs" : a}
        )
    )
print(input_label_dataset)

<MapDataset shapes: ({x: (25, 2089, 530), q: (25, 2089, 265)}, {outputs: (25, 2089, 1)}), types: ({x: tf.float32, q: tf.float32}, {outputs: tf.int32})>


##### define split fucntion into train/test/valid


In [ ]:
"""
code from https://github.com/lccasagrande/Deep-Knowledge-Tracing
"""

def split_dataset(dataset, total_size, test_fraction, val_fraction=None):
    def split(dataset, split_size):
        split_set = dataset.take(split_size)
        dataset = dataset.skip(split_size)
        return dataset, split_set

    if not 0 < test_fraction < 1:
        raise ValueError("test_fraction must be between (0, 1)")

    if val_fraction is not None and not 0 < val_fraction < 1:
        raise ValueError("val_fraction must be between (0, 1)")

    test_size = np.ceil(test_fraction * total_size)
    train_size = total_size - test_size

    if test_size == 0 or train_size == 0:
        raise ValueError(
            "The train and test datasets must have at least 1 element. Reduce the split fraction or get more data.")

    train_set, test_set = split(dataset, test_size)

    val_set = None
    if val_fraction:
        val_size = np.ceil(train_size * val_fraction)
        train_set, val_set = split(train_set, val_size)

    return train_set, test_set, val_set


## Model Codes

### try neptune.ai

In [ ]:
!pip install neptune-client

In [ ]:
env NEPTUNE_API_TOKEN=""

In [ ]:
import neptune
neptune.init('ikiyo/sandbox')

with neptune.create_experiment():
    neptune.send_metric('auc', 0.92)

### DKT code of exepriments of spaced tempral effect
https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiU6pDN7M_rAhW_A2MBHU3gD3AQFjABegQIARAB&url=https%3A%2F%2Feducationaldatamining.org%2Ffiles%2Fconferences%2FEDM2020%2Fpapers%2Fpaper_242.pdf&usg=AOvVaw1frKbFoVwfcA-a4XGEAw2i

!git clone https://github.com/thosgt/kt-algos.git

In [ ]:
!cp -r "/content/drive/My Drive/master thesis/Source code/kt-algos" ./

In [ ]:
cd kt-algos

In [ ]:
pip install -r requirements.txt

In [ ]:
!mkdir -p data/assistments12

In [ ]:
!cp "/content/drive/My Drive/master thesis/Datasets/assistment_dataset/2012-2013-data-with-predictions-4-final.csv" ./data/assistments12/data.csv

In [ ]:
!python prepare_data.py --dataset assistments12 --remove_nan_skills

In [ ]:
!python train_dkt.py --dataset assistments12 --embed_inputs --embed_size=100 --hid_size=100 --batch_size=5

### DKT in TF2 from GitHub

In [ ]:
!git clone https://github.com/lccasagrande/Deep-Knowledge-Tracing.git

In [ ]:
cd Deep-Knowledge-Tracing

In [ ]:
import pandas as pd

df = pd.read_csv("/content/Deep-Knowledge-Tracing/examples/data/ASSISTments_skill_builder_data.csv")

In [ ]:
pip install -e .[tf_gpu]

In [ ]:
!python ./run_dkt.py -f "/content/Deep-Knowledge-Tracing/examples/data/ASSISTments_skill_builder_data.csv"

### iKiYo DKT in TF2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.test.is_gpu_available())

##### build model function base

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.test.is_gpu_available())
df = pd.read_csv("/content/drive/My Drive/master thesis/Datasets/assistment_dataset/assist12_4cols_noNaNskill.csv")

# get N, M, T
num_students, num_skills, _, _ = df.nunique()
max_sequence_length=  df['user_id'].value_counts().max()
print(num_students, num_skills, max_sequence_length)
batch_size = 25
seq = df.groupby('user_id').apply(
    lambda r: (
        r['x'].values[:-1],
        r['skill_id'].values[1:],
        r['correct'].values[1:],
    )
)
assert num_students, len(seq)
seq[0]

# Step 5 - Get Tensorflow Dataset
dataset = tf.data.Dataset.from_generator(
    generator=lambda: seq,
    output_types=(tf.int32, tf.int32, tf.int32)
)
dataset.shuffle(buffer_size=num_students)

transformed_dataset  = dataset.map(
    lambda feat, skill, label: (
        tf.one_hot(feat, depth=num_skills*2), # x
        tf.one_hot(skill, depth=num_skills, axis=-1), # q
        tf.expand_dims(label, -1) # a 
    )
)

padded_sample = list(transformed_dataset.take(3).as_numpy_iterator())

for i in range(3):
  print(padded_sample[0][i].T)
  print(np.array(padded_sample[0][i].T).shape)

padding_length = max_sequence_length
padded_dataset = transformed_dataset.padded_batch(
        batch_size=batch_size,
        padding_values=(0.,0.,-1),
        padded_shapes=([padding_length, 2*num_skills], [padding_length, num_skills], [padding_length, 1]),
        drop_remainder=True
    )
padded_dataset

# to dict dataset
input_label_dataset = padded_dataset.map(
        lambda x, delta_q, a : (
             {"x" : x,
              "q" : delta_q},
             { "outputs" : a}
        )
    )
print(input_label_dataset)

In [29]:
train_dataset, test_dataset, val_dataset = split_dataset(input_label_dataset, total_size=num_batches, test_fraction=0.1, val_fraction=0.2)

In [ ]:
print(train_dataset.element_spec)
print(list(train_dataset.take(1).as_numpy_iterator()))
#print(list(test_dataset.take(1).as_numpy_iterator()))
#print(list(valid_dataset.take(1).as_numpy_iterator()))

In [25]:
# model parmaeters
hidden_units=200 
dropout_rate=0.2
embed_dim = 200
learning_rate = 0.005

num_students, num_skills, max_sequence_length

(28118, 265, 2089)

https://www.tensorflow.org/guide/keras/train_and_evaluate#passing_data_to_multi-input_multi-output_models

In [26]:
# definition of input tensor shape and layers
x = tf.keras.Input(shape=(max_sequence_length, num_skills*2), name='x')
q = tf.keras.Input(shape=(max_sequence_length, num_skills), name='q')
emb =  layers.Dense(embed_dim, trainable=False, 
                                                  kernel_initializer=tf.keras.initializers.RandomNormal(seed=777),
                                                  input_shape=(None, max_sequence_length, num_skills*2))
mask = layers.Masking(mask_value=0, input_shape=(max_sequence_length, embed_dim))
lstm =  layers.LSTM(hidden_units, return_sequences=True)
out_dropout =  layers.TimeDistributed(layers.Dropout(dropout_rate))
out_sigmoid =  layers.TimeDistributed(layers.Dense(num_skills,  activation='sigmoid'))
dot =  layers.Multiply()
# HACK: the shape of q does not fit to Timedistributed operation(may be correct?)
# dot =  layers.TimeDistributed(layers.Multiply())

reduce_sum =  layers.Dense(1, trainable=False, 
                                                  kernel_initializer=tf.keras.initializers.constant(value=1),
                                                  input_shape=(None, max_sequence_length,num_skills))
# reshape layer does not work as graph  # reshape_l = layers.Reshape((-1,6),dynamic=False)#, 
final_mask =   layers.TimeDistributed(
layers.Masking(mask_value=0, input_shape=(None, max_sequence_length,1))
,name='outputs')


# define graph
n = emb(x)
masked_n = mask(n)
h = lstm(masked_n) 
o = out_dropout(h)
y_pred = out_sigmoid(o)
y_pred = dot([y_pred, q])
# HACK: without using layer(tf.reduce) might be faster
# y_pred = reduce_sum(y_pred, axis=2)
y_pred = reduce_sum(y_pred)
outputs = final_mask(y_pred)
#  KEEP: another approach for final mask
# patch initial mask by boolean_mask(tensor, mask)
#tf.boolean_mask(y_pred, masked_n._keras_mask)
#y_pred._keras_mask=masked_n._keras_mask


# build model
model = keras.Model(inputs=[x, q], outputs=outputs)


# comile model
# set Reduction.SUM for distributed traning
model.compile(loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM),
              optimizer=tf.optimizers.SGD(learning_rate=learning_rate),
                # metrics=[tf.keras.metrics.AUC()])
              metrics=[tf.keras.metrics.AUC(),tf.keras.metrics.BinaryCrossentropy()]) # keep BCEntropyfor debug

##### train model on assist12 dataset

In [ ]:
model.predict(train_dataset.take(1))
model.summary()

###### measure the perfomace time

In [35]:
import time
class CustomCallback(keras.callbacks.Callback):
    # def on_train_begin(self, logs=None):
    #     keys = list(logs.keys())
    #     print("Starting training; got log keys: {}".format(keys))

    # def on_train_end(self, logs=None):
    #     keys = list(logs.keys())
    #     print("Stop training; got log keys: {}".format(keys))

    # def on_epoch_begin(self, epoch, logs=None):
    #     keys = list(logs.keys())
    #     print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    # def on_epoch_end(self, epoch, logs=None):
    #     keys = list(logs.keys())
    #     print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    # def on_test_begin(self, logs=None):
    #     keys = list(logs.keys())
    #     print("Start testing; got log keys: {}".format(keys))

    # def on_test_end(self, logs=None):
    #     keys = list(logs.keys())
    #     print("Stop testing; got log keys: {}".format(keys))

    # def on_predict_begin(self, logs=None):
    #     keys = list(logs.keys())
    #     print("Start predicting; got log keys: {}".format(keys))

    # def on_predict_end(self, logs=None):
    #     keys = list(logs.keys())
    #     print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        self.batch_start_time = time.time()

    def on_train_batch_end(self, batch, logs=None):
        print(F" time {time.time()-self.batch_start_time :.3f} sec")

    def on_test_batch_begin(self, batch, logs=None):
        self.batch_start_time = time.time()

    def on_test_batch_end(self, batch, logs=None):
        print(F" time {time.time()-self.batch_start_time :.3f} sec")

    # def on_predict_batch_begin(self, batch, logs=None):
    #     keys = list(logs.keys())
    #     print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    # def on_predict_batch_end(self, batch, logs=None):
    #     keys = list(logs.keys())
    #     print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))



###### tensorboard

In [ ]:
from datetime import datetime
from packaging import version

import os

In [ ]:
pip install -U tensorboard_plugin_profile

In [ ]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1)

In [ ]:
# Load the TensorBoard notebook extension.
%reload_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs

In [ ]:
!export TF_GPU_THREAD_MODE=gpu_private

###### train


In [154]:
print(hidden_units, dropout_rate, embed_dim, learning_rate,batch_size)
print(num_students, num_skills, max_sequence_length, num_batches)

model.fit(train_dataset.prefetch(6),  epochs=1,  validation_data=val_dataset)#,  callbacks=[CustomCallback()])

200 0.2 200 0.005 50
28118 265 2089 562
966/966 [==============================] - 90s 94ms/step - loss: 1341.8607 - auc_5: 0.6090 - binary_crossentropy: 0.6258 - val_loss: 2153.4209 - val_auc_5: 0.6067 - val_binary_crossentropy: 0.6006


###### evaluate with test dataset

In [ ]:
results = model.evaluate(val_dataset.prefetch(5), callbacks=[CustomCallback()])
results

In [148]:
print(hidden_units, dropout_rate, embed_dim, learning_rate,batch_size)
print(num_students, num_skills, max_sequence_length, num_batches)

model.fit(train_dataset.prefetch(6),  epochs=1,  validation_data=val_dataset)#,  callbacks=[CustomCallback()])
# model.fit(input_label_dataset.take(200).prefetch(1), epochs=1, callbacks=[tboard_callback])

# model.evaluate(input_label_dataset.take(1))

200 0.2 200 0.004 50
28118 265 2089 562
966/966 [==============================] - 90s 94ms/step - loss: 1271.3989 - auc_3: 0.6504 - binary_crossentropy: 0.5930 - val_loss: 2188.6274 - val_auc_3: 0.5910 - val_binary_crossentropy: 0.6104


In [ ]:
sample_test_data = input_label_dataset.take(1)
y_pred = model.predict(sample_test_data)

In [ ]:
# sample_test_data = test_dataset.take(1)
# y_true = list(sample_test_data.as_numpy_iterator())
y_true = y_true[0][1]['outputs']

In [ ]:
m = tf.keras.metrics.AUC()
m.update_state(y_true, y_pred)
print(m.result().numpy())
bce = tf.keras.losses.BinaryCrossentropy()
print(bce(y_true, y_pred).numpy())

In [ ]:
y_pred = model.predict(sample_test_data)
y_true_arr = y_true[0][1]['outputs'][0].T[0][0:82]
y_pred_arr = y_pred[0].T[0][0:82]
print(y_true_arr)
print(y_pred_arr)

In [ ]:
y_true_arr2 = y_true[1][1]['outputs'][0].T[0][0:15]
y_pred_arr2= y_pred[0].T[0][0:15]
print(y_true_arr2)
print(y_pred_arr2)

In [ ]:
y_true = np.concatenate((y_true_arr, y_true_arr2))
y_pred = np.concatenate((y_pred_arr, y_pred_arr2))
m = tf.keras.metrics.AUC()
m.update_state(y_true, y_pred)
print(m.result().numpy())
bce = tf.keras.losses.BinaryCrossentropy()
print(bce(y_true, y_pred).numpy())

### codes for debugging

##### checking metric and loss with raw(=masked) sample dataset

In [ ]:
history = model.evaluate(train_dataset, batch_size=3)
predictions = model.predict(train_dataset)

print("raw labels")
A = np.array([[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1]]).T
Y_pred =np.concatenate((predictions[0][0:3], predictions[1][0:5], predictions[2][0:6]))
print(F"shapes, {A.shape}, {Y_pred.shape}, len: {len(A)}")
print(A.T,Y_pred.T)


print("correct AUC")
m = tf.keras.metrics.AUC()
m.update_state(A,Y_pred)
print(m.result().numpy())
print()

print("correct metric loss for each sample")
m = tf.keras.metrics.BinaryCrossentropy()
m.update_state(A,Y_pred)
print(m.result().numpy())
print()

print("correct loss of keras loss, SUM")

# loss masked
bce = tf.keras.losses.BinaryCrossentropy()
print(bce(A,Y_pred).numpy())

print("loss SUM")
bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
print(bce(A,Y_pred).numpy())
print(bce.get_config())

# print()
# print("loss from inside model")
# bce = model.loss
# print("SUM")
# print(model.loss(A, Y_pred).numpy())
# print(model.loss.get_config())

##### build model class base( WARNING: run only on numpy input data )

In [ ]:
# FIXME:  evaluate cause an error below
# step=1 does not work
"""
WARNING:tensorflow:8 out of the last 15 calls to <function Model.make_test_function.<locals>.test_function at 0x7f55ca8a7a60>
 triggered tf.function retracing. 
 Tracing is expensive and the excessive number of tracings could be due to 
 (1) creating @tf.function repeatedly in a loop,
 (2) passing tensors with different shapes,
  (3) passing Python objects instead of tensors. 
  For (1), please define your @tf.function outside of the loop.
   For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. 
   For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args 
   and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
"""
class DKT(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.x = tf.keras.Input(shape=(max_sequence_length, num_skills*2), name='x')
    self.q = tf.keras.Input(shape=(max_sequence_length, num_skills), name='q')
    self.inputs = [self.x , self.q]
    # self.x = tf.keras.Input(shape=(None, max_sequence_length, num_skills*2), name='inputs')
    # self.q = tf.keras.Input(shape=(None, max_sequence_length, num_skills), name='labels')

    self.emb =  layers.Dense(embed_dim, trainable=False, 
                                                        kernel_initializer=tf.keras.initializers.RandomNormal(seed=777),
                                                        input_shape=(None, max_sequence_length, num_skills*2))
    self.mask = layers.Masking(mask_value=0)
    self.lstm =  layers.LSTM(hidden_units, return_sequences=True)
    self.out_dropout =  layers.TimeDistributed(layers.Dropout(dropout_rate))
    self.out_sigmoid =  layers.TimeDistributed(layers.Dense(num_skills,  activation='sigmoid'))

    # HACK: the shape of q does not fit to Timedistributed operation(may be faster?)
    self.dot =  layers.Multiply()
    # self.dot =  layers.TimeDistributed(layers.Multiply())

    #self.reduce_sum = tf.reduce_sum
    #self.transpose = tf.transpose

    self.reduce_sum =  layers.Dense(1, trainable=False, 
                                                        kernel_initializer=tf.keras.initializers.constant(value=1),
                                                        input_shape=(None, max_sequence_length,num_skills))
    # reshape layer does not work as graph 
    # self.reshape_l = layers.Reshape((-1,3),dynamic=False)#, 
    #                                                      # input_shape=(None,max_sequence_length,num_skills,))
    self.final_mask =   layers.TimeDistributed(
      layers.Masking(mask_value=0, input_shape=(None, max_sequence_length,1))
    )


  def call(self, inputs):
    self.inputs= inputs
    x , q = self.inputs['x'], self.inputs["q"]
    # x = inputs
    # x, q = inputs
    # self.x, self.q = inputs[0], inputs[1]
    # n = self.emb(x)
    n = self.emb(self.x)
    masked_n = self.mask(n)
    h = self.lstm(masked_n) 
    o = self.out_dropout(h)
    y_pred = self.out_sigmoid(o)

    #
    # q = tf.slice(x, [0, 0, 0], [-1, -1, num_skills])  
    # y_pred = self.dot([y_pred, q])
    y_pred = self.dot([y_pred, self.q])
    # y_pred = self.reduce_sum(y_pred, axis=2)
    y_pred = self.reduce_sum(y_pred)
    #y_pred = self.reshape_l(y_pred)
    #y_pred = self.transpose()
    y_pred = self.final_mask(y_pred)
    #y_pred._keras_mask=masked_n._keras_mask
    return y_pred

# Create an instance of the model
model = DKT()

# Configuration of model
# Set "SUM" to loss reduction in order to use distributed training
model.compile(loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM),
                optimizer=tf.optimizers.SGD(learning_rate=learning_rate),
                metrics=[tf.keras.metrics.AUC()])
                # metrics=[tf.keras.metrics.AUC(),tf.keras.metrics.BinaryCrossentropy()]) # keep for debug loss

In [ ]:
one_data = input_label_dataset.take(1)
a,b = one_data[0],one_data[1]

In [ ]:
# initialize and check model structure
predictions = model([one_hot_input, one_hot_q_t])
# predictions = model(input_label_dataset)
model.summary()

In [ ]:
# check the shapes of outputs
q = one_hot_q_t
n = model.emb(one_hot_input)
print(tf.shape(n))
masked_n = model.mask(n)
print(tf.shape(masked_n))
h = model.lstm(masked_n) 
print(tf.shape(h))
o = model.out_dropout(h)
y_pred = model.out_sigmoid(o)
print(tf.shape(y_pred))
y_pred = model.dot([y_pred, q])
print(tf.shape(y_pred))
y_pred = model.reduce_sum(y_pred)
print(tf.shape(y_pred))
y_pred = model.mask(y_pred)
print(tf.shape(y_pred))


##### Check prediction and evaluation

In [ ]:
# predictions = model([one_hot_input, one_hot_q_t])
# predictions = model(one_hot_input)
predictions = model(train_dataset)

# check masking works correctly
# print(predictions._keras_mask.numpy())
# print(padded_inputs)
print()

# check shape of predicted vector
print("predcitions shape:")
print(predictions.shape) #num_student, T, M(num_skill or num_problem)
print(predictions, padded_labels)
print()

# TODO: avoid reshape before feeding
padded_labels = tf.reshape(padded_labels,(len(raw_labels),6,1))
#padded_labels = tf.transpose(padded_labels)
print(padded_labels.shape)
model.evaluate([one_hot_input, one_hot_q_t], padded_labels , verbose =2, steps=1)
# model.evaluate(one_hot_input, padded_labels , verbose =2, steps=1)

##### Check the metrics and loss works correctly

In [ ]:
predictions = model.predict(train_dataset)

m = tf.keras.metrics.AUC()
# print("sample dataset")
# m.update_state([0, 0, 1, 1], [0, 0.5, 0.3, 0.9])
# print(m.result().numpy())


print("orignal built-in")
# reshaped_labels = tf.reshape(padded_labels, [6,3])
# reshaped_predictions = tf.reshape(predictions, [6,3])
m.update_state(padded_labels,predictions)
print(m.result().numpy())
# print(len(padded_labels))
"""
print("each instance with masking")
m = tf.keras.metrics.AUC()
print([1, 1, 1], tf.transpose(predictions[0][0:3]))
m.update_state([[1, 1, 1]], tf.transpose(predictions[0][0:3]))
print(m.result().numpy())

m = tf.keras.metrics.AUC()
print(padded_labels[1][0:5],predictions[1][0:5])
m.update_state(padded_labels[1][0:5],predictions[1][0:5])
print(m.result().numpy())

m = tf.keras.metrics.AUC()
print(padded_labels[2][0:6],predictions[2][0:6])
m.update_state(padded_labels[2][0:6],predictions[2][0:6])
print(m.result().numpy())
"""
print("all with masking")
A =  [1, 1, 1, 0, 0, 0, 1, 1,  0, 1, 1, 1, 0, 1]
Y_pred =np.concatenate((predictions[0][0:3], predictions[1][0:5], predictions[2][0:6]))
# A = tf.reshape(padded_labels, [-1])
# Y_pred = predictions
print(A,Y_pred.T)
print("correct AUC")
m = tf.keras.metrics.AUC()
m.update_state(A,Y_pred)
print(m.result().numpy())
print()


# debug for loss function 
"""
# sample 
print((np.log(0.4)*3 + np.log(0.6)*1)/4)
print(raw_labels, predictions[0][0:3])

# normal loss
bce = tf.keras.losses.BinaryCrossentropy()
print(bce(padded_labels[0][0:3],predictions[0][0:3]).numpy())
"""
#print(np.log(0.5)*3)
# loss SUM
print("original built-in")
bce = tf.keras.losses.BinaryCrossentropy()
print(bce(A,Y_pred).numpy())
print("without mask")
print(bce(padded_labels,predictions).numpy())
print("loss SUM")
bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
print(bce(A,Y_pred).numpy())
print("without mask")
print(bce(padded_labels,predictions).numpy())
#total = bce(np.zeros(3), np.ones(3))*3 - np.log(0.5)*3
#total/6

##### check how masking is probagated

In [ ]:
# check how masking is probagated
q = one_hot_q_t
n = model.emb(one_hot_input)
masked_n = model.mask(n)
h = model.lstm(masked_n) 
print(h._keras_mask)
o = model.out_dropout(h)
print(o._keras_mask)
y_pred = model.out_sigmoid(o)
print(y_pred._keras_mask)
y_pred = model.dot([y_pred, q])
print(y_pred._keras_mask)
y_pred = model.reduce_sum(y_pred)
print(y_pred._keras_mask)
y_pred = model.mask(y_pred)
print(y_pred._keras_mask)

##### fit

In [ ]:
model.fit([one_hot_input, one_hot_q_t], padded_labels, epochs=50 ,verbose =1)

In [ ]:
predictions = model([one_hot_input, one_hot_q_t]).numpy()
print(predictions)
print(padded_labels)
print(raw_labels)
model.evaluate([one_hot_input, one_hot_q_t], padded_labels ,verbose =2)
